In [1]:
import pandas
import splinter
from bs4 import BeautifulSoup
import time
import selenium

In [2]:
url = 'https://www.drivethrurpg.com/en/browse?ruleSystem=44826'

In [3]:
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# #browser = splinter.Browser()
# browser = selenium.webdriver.Chrome()
# browser.get(url)
# time.sleep(3)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(2)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# soup = BeautifulSoup(browser.page_source, parser="html")
# browser.quit()

In [4]:

browser = splinter.Browser()
browser.visit(url)
time.sleep(3)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
soup = BeautifulSoup(browser.html, parser="html")
browser.quit()

In [5]:
divs = soup.find_all("div", class_="title-strip-carousel-titles-list")
# the newest div is the second title-strip
div = divs[1]

In [6]:
cards = div.find_all("obs-title-strip-card")
for card in cards:
    anchor = card.find("a")
    print(anchor["href"])

/en/product/480148/world-cartographic-world-map-design-templates?src=newest_recent
/en/product/480025/eldritch-maps-the-forest-altar?src=newest_recent
/en/product/479931/europa-theater-bundle?src=newest_recent
/en/product/479847/rangers-in-campaign-6?src=newest_recent
/en/product/479360/la-campana-del-ranger-6?src=newest_recent
/en/product/474088/free-map-dungeon-village?src=newest_recent
/en/product/478987/quasar-trilogy-digital-bundle?src=newest_recent
/en/product/476224/power-outage?src=newest_recent


In [7]:
l = soup.find("ul", class_="list-clean")
for item in l.find_all("li"):
    a = item.find("a", class_="u-dis-block")
    if a != None:
        print(a['href'])

In [8]:
def match_attr(tag):
    return tag.name == "ul" and tag['class'] == ['list-clean'] and not ("_ngcontent-serverapp-c1510817403" in tag.attrs)

In [9]:
def match_obs(tag):
    return tag and tag.name == "obs-result-list" and ("_nghost-serverapp-c1510817403" in tag.attrs)

In [12]:
urls = []
obs = soup.find(match_obs)
ul = obs.find("ul", class_="list-clean")
lis = ul.find_all("li")
for li in lis:
    a = li.find("a")
    if 'href' in a.attrs:
        urls.append("https://www.drivethrurpg.com" + a['href'])
        print(a['href'])

/en/product/370454/runequest-starter-set
/en/product/475104/going-underground-rivers-of-london-rpg-scenario
/en/product/469816/call-of-cthulhu-arkham
/en/product/260810/call-of-cthulhu-starter-set
/en/product/431991/basic-roleplaying-universal-game-engine
/en/product/462154/heinrich-s-call-of-cthulhu-guide-to-carcosa
/en/product/476836/alone-against-nyarlathotep
/en/product/476571/kane-s-tone
/en/product/446091/japan-empire-of-shadows-a-call-of-cthulhu-sourcebook-for-1920s-imperial-japan
/en/product/476224/power-outage
/en/product/391974/viral-a-modern-call-of-cthulhu-scenario
/en/product/461171/alone-against-the-static
/en/product/480227/the-million-flavoured-ones
/en/product/418884/the-comae-engine
/en/product/476312/in-darkness-we-wait
/en/product/385006/korean
/en/product/468943/book-of-schemes
/en/product/191475/mythras
/en/product/24384/basic-roleplaying
/en/product/386943/korean-the-world-of-cliche
/en/product/365897/korean-100-chronicle-of-100-hours
/en/product/478643/the-oracl

In [83]:
browser = splinter.Browser()
titles = []
for page in urls[0:2]:
    print(f"visiting {page}")
    browser.visit(page)
    soup2 = BeautifulSoup(browser.html, parser="html")
    title = soup2.find("h2").text.strip()
    page_count = int(soup2.find("div", class_="product-detail-tile-4").find_all("p")[2].text.strip())
    products = soup2.find("obs-product-format-options").find_all("obs-product-format-option-selection-tile")
    product_dicts = []
    for product in products:
        product_name = product.find("p", class_="product-tile-text-highlight").text.strip()
        price = float(product.find_all("p", class_="product-tile-text-highlight")[-1].text.strip().replace("$",""))
        product_dicts.append({"product":product_name, "price":price})
    published_date = soup2.find("div", class_="product-detail-tile-5").find("p", class_="u-text-bold").text.strip()
    format = soup2.find("p", attrs={"data-codeid":"format"}).parent.next_sibling.find('p').text.strip()
    author_block = soup2.find("obs-publisher-or-creators", listonly="authors")
    authors = []
    for a in author_block.find_all("a"):
        authors.append(a.text.strip().replace(",", ""))
    artist_block = soup2.find("obs-publisher-or-creators", listonly="artists")
    artists = []
    for a in artist_block.find_all("a"):
        artists.append(a.text.strip().replace(",", ""))
    system = soup2.find("p", attrs={"data-codeid":"ruleSystem"}).parent.next_sibling.find("p").text.strip()
    modified_date = soup2.find("p", attrs={"data-codeid":"dateModified"}).parent.next_sibling.find("p").text.strip()
    titles.append({
        "title":title,
        "page_count":page_count,
        "products":product_dicts,
        "published_date":published_date,
        "format":format,
        "authors":authors,
        "artists":artists,
        "system":system,
        "modified_date":modified_date
    })
browser.quit()
titles

visiting https://www.drivethrurpg.com/en/product/370454/runequest-starter-set
visiting https://www.drivethrurpg.com/en/product/475104/going-underground-rivers-of-london-rpg-scenario


[{'title': 'RuneQuest Starter Set',
  'page_count': 180,
  'products': [{'product': 'PDF', 'price': 14.99}],
  'published_date': '11/10/21',
  'format': 'PDF',
  'authors': ['Greg Stafford', 'Jeff Richard', 'Jason Durall'],
  'artists': ['Ossi Hiekkala',
   'Andrey Fetisov',
   'Jon Hodgson',
   'Kalin Kadiev',
   'Roman Kisyov',
   'Michelle Lockamy',
   'Jan Pospisil',
   'Scott Purdy',
   'Naomi Robinson',
   'Simon Roy',
   'Chris Waller',
   'Jon Hodgson',
   'Scott Purdy',
   'Roman Kisyov',
   'Dimitrina Angelska'],
  'system': 'RuneQuest, BRP (Basic Roleplaying)',
  'modified_date': 'Nov 9, 2021'},
 {'title': 'Going Underground - Rivers of London RPG Scenario',
  'page_count': 40,
  'products': [{'product': 'PDF', 'price': 6.99},
   {'product': 'Softcover, Premium Color Book (8.25" x 10.75")',
    'price': 20.99},
   {'product': 'Softcover, Standard Color Book (8.25" x 10.75")',
    'price': 14.99},
   {'product': 'PDF + Softcover, Premium Color Book (8.25" x 10.75")',
    'pri

In [13]:
test_url = urls[0]
browser = splinter.Browser()
browser.visit(test_url)
soup2 = BeautifulSoup(browser.html, parser="html")
browser.quit()

In [17]:
page_count = int(soup2.find("div", class_="product-detail-tile-4").find_all("p")[2].text.strip())
page_count

180

In [25]:
products = soup2.find("obs-product-format-options").find_all("obs-product-format-option-selection-tile")
product_dicts = []
for product in products:
    title = product.find("p", class_="product-tile-text-highlight").text.strip()
    price = float(product.find_all("p", class_="product-tile-text-highlight")[-1].text.strip().replace("$",""))
    product_dicts.append({"title":title, "price":price})
product_dicts

[{'title': 'PDF', 'price': 14.99}]

In [29]:
published_date = soup2.find("div", class_="product-detail-tile-5").find("p", class_="u-text-bold").text.strip()
published_date

'11/10/21'

In [39]:
def format_row_test(tag):
    if not tag:
        return False
    if tag.name != "tr":
        return False
    p = tag.find("p")
    if p == None:
        return False
    if p.text.strip() != "Format":
        return False
    return True

In [41]:
format_row = soup2.find(format_row_test)
format_row

<tr _ngcontent-serverapp-c2702281686=""><td _ngcontent-serverapp-c2702281686=""><p _ngcontent-serverapp-c2702281686="" class="u-text-bold" data-codeid="format"> Format </p></td><td _ngcontent-serverapp-c2702281686=""><p _ngcontent-serverapp-c2702281686=""> PDF <!-- --></p></td></tr>

In [48]:
p = soup2.find("p", attrs={"data-codeid":"format"})
format = p.parent.parent.find_all("td")[1].find('p').text.strip()

'PDF'

In [57]:
author_block = soup2.find("obs-publisher-or-creators", listonly="authors")
authors = []
for a in author_block.find_all("a"):
    authors.append(a.text.strip().replace(",", ""))
authors

['Greg Stafford', 'Jeff Richard', 'Jason Durall']

In [66]:
artist_block = soup2.find("obs-publisher-or-creators", listonly="artists")
artists = []
for a in artist_block.find_all("a"):
    artists.append(a.text.strip().replace(",", ""))
artists

['Ossi Hiekkala',
 'Andrey Fetisov',
 'Jon Hodgson',
 'Kalin Kadiev',
 'Roman Kisyov',
 'Michelle Lockamy',
 'Jan Pospisil',
 'Scott Purdy',
 'Naomi Robinson',
 'Simon Roy',
 'Chris Waller',
 'Jon Hodgson',
 'Scott Purdy',
 'Roman Kisyov',
 'Dimitrina Angelska']

In [68]:
system = soup2.find("p", attrs={"data-codeid":"ruleSystem"}).parent.next_sibling.find("p").text.strip()
system

'RuneQuest, BRP (Basic Roleplaying)'

In [69]:
modified_date = soup2.find("p", attrs={"data-codeid":"dateModified"}).parent.next_sibling.find("p").text.strip()
modified_date

'Nov 9, 2021'

In [71]:
for h2 in soup2.find_all("h2"):
    print(h2.text)

RuneQuest Starter Set
Magic, Myth, and Might in a Box!
What is Roleplaying?
What's Inside
Rules & Dice
SoloQuest
Scenarios
Maps & Handouts
What's Next?


In [73]:
soup2.find("h2").text.strip()

'RuneQuest Starter Set'

In [79]:
url

'https://www.drivethrurpg.com/en/browse?ruleSystem=44826'

In [77]:
soup2.find("obs-product-format-options").attrs

AttributeError: 'NoneType' object has no attribute 'attrs'

In [80]:
page

'https://www.drivethrurpg.com/en/product/475104/going-underground-rivers-of-london-rpg-scenario'